In [ ]:
import os
import numpy as np
import soundfile as sf # for opening FLAC
import librosa # for MFCC
from scipy.signal import spectrogram

path_dir = ""

for filename in os.listdir(path_dir):
    file_path = os.path.join(path_dir, filename)
    audio_data, sample_rate = sf.read(file_path)

    frequencies, times, spectrogram_data = spectrogram(audio_data, sample_rate)

